## 1,2. Загрузка датасета в Python

In [51]:
import pandas as pd
import numpy as np
MyFile = pd.read_csv("/Users/Anton/Desktop/Учеба/Тексты/smsspamcollection/SMSSpamCollection", 
                 delimiter="\t", names=("isSpam", "smsText"))

3.Spam в колонке isSpam помечается как 1, ham как 0. Создание списка текстов сообщений и списка меток класса.

In [52]:
for i in range(0,MyFile.isSpam.size):
    MyFile.isSpam[i] = int(MyFile.isSpam[i] == "spam")
         

In [53]:
texts = [MyFile.smsText[i] for i in range(0,MyFile.smsText.size)]
tags = [MyFile.isSpam[i] for i in range(0,MyFile.isSpam.size)]

4.Получение из списка текстов матрицу признаков Х

In [54]:
from sklearn.feature_extraction.text import CountVectorizer


In [55]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

5.Получение оценки качества классификации текстов. Полученое значение 0.932640298361

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

cls = LogisticRegression()
res = cross_val_score(cls, X, tags, scoring="f1", cv=10)
print (np.mean(res))

0.932640298361


6.Обучение классификатора на всей выборке и прогнозирование класса на примерах. Из результата видно, что классификатор отнес первые два сообщения к спаму, остальные три к хаму.

In [57]:
cls.fit(X, tags)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [58]:
sample = vectorizer.transform(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                               "FreeMsg: Txt: claim your reward of 3 hours talk time", 
                               "Have you visited the last lecture on physics?",
                               "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$" ,
                              "Only 99$"])
prediction = cls.predict(sample)
print(prediction)

[1 1 0 0 0]


7.Оценка качества с добавлением в признаки n-граммы.

In [59]:
vectorizer1 = CountVectorizer(ngram_range=(2,2))
X1 = vectorizer1.fit_transform(texts)
res1 = cross_val_score(cls, X1, tags, scoring="f1", cv=10)

vectorizer2 = CountVectorizer(ngram_range=(3,3))
X2 = vectorizer2.fit_transform(texts)
res2 = cross_val_score(cls, X2, tags, scoring="f1", cv=10)

vectorizer3 = CountVectorizer(ngram_range=(1,3))
X3 = vectorizer3.fit_transform(texts)
res3 = cross_val_score(cls, X3, tags, scoring="f1", cv=10)

print str(round(np.mean(res1),2)) + " " + str(round(np.mean(res2),2)) + " " + str(round(np.mean(res3),2))

0.82 0.73 0.93


8.Повторение эксперимента из пункта 7, но с использованием наивного Байеса. Можно заметить резкое снижение качества, связанное с нехваткой статистики по биграммам и триграммам.

In [60]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

vectorizer4 = CountVectorizer(ngram_range=(2,2))
X4 = vectorizer4.fit_transform(texts)
res4 = cross_val_score(nb, X4, tags, scoring="f1", cv=10)

vectorizer5 = CountVectorizer(ngram_range=(3,3))
X5 = vectorizer5.fit_transform(texts)
res5 = cross_val_score(nb, X5, tags, scoring="f1", cv=10)

vectorizer6 = CountVectorizer(ngram_range=(1,3))
X6 = vectorizer6.fit_transform(texts)
res6 = cross_val_score(nb, X6, tags, scoring="f1", cv=10)

print str(round(np.mean(res4),2)) + " " + str(round(np.mean(res5),2)) + " " + str(round(np.mean(res6),2))

0.65 0.38 0.89


9.Использование признаков Tf*idf из TfidfVectorizer на униграммах в логистической регрессии. Качество ухудшилось, 0.85 против 0.93 на CountVectorizer()

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(ngram_range=(1,1))
X_tfidf = tfidf.fit_transform(texts)
cls1 = LogisticRegression()

res7 = cross_val_score(cls1, X_tfidf, tags, scoring="f1", cv=10)

print(round(np.mean(res7),2))


0.85


### 10.Тестирование различных параметров, классификаторов и способов обработки текста

a) Pipeline c данными параметрами продемонстрировал качество на кросс-валидациях в 0,951

In [62]:
from sklearn.pipeline import Pipeline

text_processing_pipeline = Pipeline([
        ('Vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
        ('Classifier', LogisticRegression(C = 1000))
    ])

res8 = cross_val_score(text_processing_pipeline,texts, tags, scoring="f1", cv=10)
print(np.mean(res8))

0.951724244087


б) Использование стохастического градиентного спуска в качестве классификатора в лучшем случае(при данном наборе параметров) продемонстрировало качество примерно равное 0,94

In [63]:
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
reg = SGDClassifier(loss="log", penalty="l2")
resss = cross_val_score(reg, X, tags, scoring="f1", cv=10)
print (np.mean(resss))

0.938020158284


в) Использование дисижен три в качестве классификатора показало качество примерно равное 0.88

In [64]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
resss1 = cross_val_score(clf, X, tags, scoring="f1", cv=10)
print (np.mean(resss1))

0.882366100781


г) Фильтрация текстов от стоп-слов не помогла повысить качество, которое только ухудшилось.

In [65]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /Users/Anton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
filtered = texts
for i in range(0,5572):
    stri = ""
    for w in texts[i].split(" "):
        if w.lower() not in stopwords.words('english'):
            stri = stri + w + " "
    filtered[i] = stri
            

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [67]:
X_filtered = vectorizer.fit_transform(filtered)

cls_filtered = LogisticRegression()
res_filtered = cross_val_score(cls_filtered, X_filtered, tags, scoring="f1", cv=10)
print (np.mean(res_filtered))

0.928275521277


д) Использование стэмминга увеличило качество модели до 0,9357.

In [68]:
MyFile_stem = pd.read_csv("/Users/Anton/Desktop/Учеба/Тексты/smsspamcollection/SMSSpamCollection", 
                 delimiter="\t", names=("isSpam", "smsText"),encoding='utf-8')
    
texts_stem = [MyFile_stem.smsText[i] for i in range(0,MyFile_stem.smsText.size)]


from nltk.stem import PorterStemmer
ps = PorterStemmer()
stem = texts
for i in range(0,5572):
    stri = ""
    for w in texts_stem[i].split(" "):
        stri = stri + ps.stem(w) + " "
    stem[i] = stri

In [69]:
X_stem = vectorizer.fit_transform(stem)

cls_stem = LogisticRegression()
res_stem = cross_val_score(cls_stem, X_stem, tags, scoring="f1", cv=10)
print (np.mean(res_stem))

0.935769174389


е) Использование лемматизации увеличило качество модели до 0,9357.

In [72]:

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

lemma = texts
for i in range(0,5572):
    stri = ""
    for w in texts[i].split(" "):
        stri = stri + lemmatizer.lemmatize(w) + " "
    lemma[i] = stri

In [73]:
X_lemma = vectorizer.fit_transform(lemma)

cls_lemma = LogisticRegression()
res_lemma = cross_val_score(cls_lemma, X_lemma, tags, scoring="f1", cv=10)
print (np.mean(res_lemma))

0.935736895793


## Наблюдения и выводы из задания

1) Отсутствие статистики по биграммам и триграммам сказывается на качестве классификатора, особенно страдают нелинейные классификаторы. В то же время при использовании биграмм и триграмм вместе с униграммами в линейном классификаторе, качество существенно не пострадает, т.к. из-за регуляризации данный классификатор не склонен сильно переобучаться на этих признаках.

2) Подбор параметров классификатора способен улучшить качество классификатора

3) Различные классификаторы в большинстве своем демонстрируют различное качество

4) Фильтрация текстов от стоп-слов не всегда гарантирует улучшение качества классификатора